In [47]:
import backtrader as bt
from datetime import datetime
import matplotlib

In [48]:
data = bt.feeds.YahooFinanceCSVData(
        dataname='./datasets/MSFT.csv',
        fromdate=datetime(2021, 9, 1),
        todate=datetime(2022, 8, 31),
        reverse=False)

In [49]:
class SMAStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])
        if self.order:
            return
        if not self.position:
            if self.dataclose[0] > self.sma[0]:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
        else:
            if self.dataclose[0] < self.sma[0]:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()

In [50]:
class RSIStrategy(bt.Strategy):
    params = (
        ('maperiod', 10),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.rsi = bt.indicators.RSI(
            self.datas[0], period=self.params.maperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])
        if self.order:
            return
        if not self.position:
            if self.rsi[-1] > 30 and self.rsi[0] < 30:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
        else:
            if self.rsi[-1] < 70 and self.rsi[0] > 70:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()

In [51]:
cerebro = bt.Cerebro()
cerebro.addstrategy(RSIStrategy)
cerebro.broker.setcash(100000.0)
cerebro.broker.setcommission(commission=0.001)
cerebro.addsizer(bt.sizers.FixedSize, stake=10)
cerebro.adddata(data)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2021-09-16, Close, 302.68
2021-09-17, Close, 297.38
2021-09-20, Close, 291.86
2021-09-21, Close, 292.35
2021-09-22, Close, 296.10
2021-09-23, Close, 297.07
2021-09-24, Close, 296.86
2021-09-27, Close, 291.73
2021-09-28, Close, 281.17
2021-09-28, BUY CREATE, 281.17
2021-09-29, BUY EXECUTED, Price: 282.73, Cost: 2827.30, Comm 2.83
2021-09-29, Close, 281.64
2021-09-30, Close, 279.58
2021-10-01, Close, 286.70
2021-10-04, Close, 280.76
2021-10-05, Close, 286.36
2021-10-06, Close, 290.68
2021-10-07, Close, 292.40
2021-10-08, Close, 292.40
2021-10-11, Close, 291.79
2021-10-12, Close, 290.45
2021-10-13, Close, 293.85
2021-10-14, Close, 300.24
2021-10-15, Close, 301.68
2021-10-18, Close, 304.74
2021-10-19, Close, 305.67
2021-10-19, SELL CREATE, 305.67
2021-10-20, SELL EXECUTED, Price: 306.64, Cost: 2827.30, Comm 3.07
2021-10-20, OPERATION PROFIT, GROSS 239.10, NET 233.21
2021-10-20, Close, 304.86
2021-10-21, Close, 308.18
2021-10-22, Close, 306.59
2021-10-25,

In [52]:
%matplotlib inline
cerebro.plot()

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 5 Axes>]]